### Create upazilla list from text file

In [30]:
class Upazilla:
    '''Model class for Upazilla'''
    def __str__(self):
        return (upazilla.upazilla_name_en + '/'
               +upazilla.upazilla_name_bn + ','
               +upazilla.zilla_name_en + '/'
               +upazilla.zilla_name_bn)
    pass
def get_upazillas(input_handle):
    '''Returns a list of all upazillas'''
    upazillas = []
    with open(input_handle, 'r', encoding="utf8") as file:
        lines = file.readlines()
    for line in lines:
        text = line.split(',')
        upazilla = Upazilla()
        upazilla.upazilla_name_en = text[0]
        upazilla.upazilla_name_bn = text[1]
        upazilla.zilla_name_en = text[2]
        upazilla.zilla_name_bn = text[3][:-1] # stray newline character
        upazillas.append(upazilla)
    return upazillas

### Create article list from text file

In [31]:
import re
import datetime
class Article:  
    def __init__(self, text):
        '''create an Article object from newline seperated text'''
        lines = text.split('\n')
        self.site = lines[0]
        self.link = lines[1]
        keys = re.split(',? ', lines[2])
        # only for road-accident
#         if len(keys) > 2:
#             keys = [keys[0]+'_'+keys[1], keys[2]+'_'+keys[3]]
#         else:
#             keys = [keys[0]+'_'+keys[1]]
        self.keywords = keys
        self.headline = lines[3]
        textdate = lines[4]
        if textdate[2] == '-':
            # date is in dd-mm-yyyy format
            self.date = datetime.date(int(textdate[-4:]), int(textdate[3:5]), int(textdate[:2]))
        else:
            # date is in yyyy-mm-dd format
            self.date = datetime.date(int(textdate[:4]), int(textdate[5:7]), int(textdate[-2:]))
        if len(lines) > 6:
            self.upazillas = lines[-2][5:].split(', ')
        self.news = lines[-1]
    
    def __str__(self):
        '''Converts back to String'''
        return (self.site + '\n'
               + self.link + '\n'
               + ', '.join(self.keywords) + '\n'
               + self.headline + '\n'
               + str(self.date) + '\n'
               + self.news)
    def __cmp__(self, other):
        if self.headline == other.headline and self.news == other.news:
            return 0
        elif self.date > other.date:
            return -1
        else:
            return 1
    def __lt__(self, other):
        return self.date > other.date
    def __le__(self, other):
        return self.date >= other.date
    def __eq__(self, other):
        return self.headline == other.headline and self.news == other.news
    def __ne__(self, other):
        return self.news != other.news
    def __gt__(self, other):
        return self.date < other.date
    def __ge__(self, other):
        return self.date <= other.date
        

def get_articles(path):
    articles = []
    with open(path, 'r', encoding = 'utf-8') as file:
        raw = file.read().split('\n######\n')  
    for text_block in raw:
        article = Article(text_block)
        # prevent duplicates
        if article not in articles:
            articles.append(article)
    return articles
        
# path = 'E:Dropbox/Sachetan_Crawl/prothom-alo/prothom_alo_theft_step-1.txt'
# output = 'E:Dropbox/Sachetan_Crawl/prothom-alo/prothom_alo_theft_step-2-date-sorted.txt'
# articles = get_articles(path)
# with open(output, 'w', encoding = 'utf-8') as file:
#     file.write('\n######\n'.join([str(article) for article in sorted(articles)]))

### Search and add upazilla to articles

In [44]:
def upazillas_from_sentence(upazillas, sentence):
    print(sentence)
    tagged_upazillas = set()
    words = sentence.split()
    for i in range(len(words)):
        word = words[i].replace('(', '').replace(')', '').replace(':', '').replace(';', '').replace('&', '')
        for upazilla in upazillas:
            name_bn = upazilla.upazilla_name_bn
            if name_bn.startswith('রা'):
                print(word, upazilla.upazilla_name_bn)
            if ' ' in upazilla.upazilla_name_bn and i < len(words) - 1:
                # two letter upazilla name
                uz_name_fragments = upazilla.upazilla_name_bn.split();
                if words[i].startswith(uz_name_fragments[0]) and words[i+1].startswith(uz_name_fragments[1]):
                        tagged_upazillas.add(upazilla.upazilla_name_en.replace(' ', '_'))
            elif word.startswith(upazilla.upazilla_name_bn):
                print('bingo')
                tagged_upazillas.add(upazilla.upazilla_name_en)
    return tagged_upazillas
                
def upazilla_tagger(article):
    upazillas = get_upazillas('E:/Dropbox/Sachetan_Crawl/201314upazilawithzila.txt')
    print(article.headline)
    print(article.news)
    print('processing...')
    article.upazillas = set()
    # look for upazilla in the headline
    tagged_upazillas = upazillas_from_sentence(upazillas, article.headline)
    # if the upazilla is found in the headline, then extract it and finish
    if tagged_upazillas:
        article.upazillas.update(tagged_upazillas)
        return article
    else:
        # if upazilla is not in headline, search in the text
        sentences = article.news.split('। ')
        checked_sentences = set()
        for i in range(len(sentences)):
            if i in checked_sentences:
                print('skipping: ' + str(i))
                continue
            for keyword in article.keywords:
                keyword = keyword.replace('_', ' ')        
                if keyword in sentences[i]:
                    '''found the keyword in current sentence'''
                    print('>> keyword hit: ' + sentences[i])
                    tagged_upazillas = upazillas_from_sentence(upazillas, sentences[i])
                    checked_sentences.add(i)
                    if tagged_upazillas:
                        '''upazilla hit on the same sentence'''
                        article.upazillas.update(tagged_upazillas)
                        '''track the sentences already processed'''
                        print('>>>> Upazilla hit')
                        break
                    else:
                        '''look at unexplored neighboring sentences
                           proximity denotes the number of sentences searched forward and backward
                           if proximity = 2, then sentences i-1, i+1, i-2, i+2 will be searched consecutively
                           for upazilla information'''
                        print('>>>>>> proximity check')
                        proximity = 2
                        for p in range(proximity + 1):
                            if i-p not in checked_sentences and i-p >= 0:
                                print('proximity ' + str(-p) + ': ' + sentences[i-p])
                                tagged_upazillas = upazillas_from_sentence(upazillas, sentences[i-p])
                                checked_sentences.add(i-p)
                                if tagged_upazillas:
                                    article.upazillas.update(tagged_upazillas)
                                    break
                            elif i+p not in checked_sentences and i+p < len(sentences):
                                    print('proximity ' + str(p) + ': ' + sentences[i+p])
                                    tagged_upazillas = upazillas_from_sentence(upazillas, sentences[i+p])
                                    checked_sentences.add(i+p)
                                    if tagged_upazillas:
                                        article.upazillas.update(tagged_upazillas)
                                        break                                    
    return article

        
path = 'E:Dropbox/Sachetan_Crawl/Step_2/bhorer_kagoj_mugging_step-2-date-sorted.txt'
articles = get_articles(path)
upazillas = get_upazillas('E:/Dropbox/Sachetan_Crawl/201314upazilawithzila_charfixed.txt')
#for upazilla in upazillas:
#    print(upazilla)
up = upazilla_tagger(articles[15])
print(up.upazillas)

রায়পুরে বাবা-ছেলেকে কুপিয়ে ৫ লাখ টাকা ছিনতাই
শংকর মজুমদার, রায়পুর (লক্ষীপুর) থেকে : উপজেলায় এলএম পাইলট মডেল উচ্চ বিদ্যালয়ের অবসরপ্রাপ্ত শিক্ষক মোস্তফা কামাল (৬০) ও ছেলে কাজলকে (৩৫) কুপিয়ে প্রায় ৫ লাখ ৪৮ হাজার টাকা নিয়ে গেছে ছিনতাইকারীরা। গত শনিবার রাতে সাবেক পৌরসভা কার্যালয়ের সামনে এ ঘটনা ঘটে। এ ব্যাপারে আহত ব্যবসায়ী মোস্তফা কামাল বাদী হয়ে রায়পুর থানায় একটি অভিযোগ করেন। পুলিশ ও স্থানীয়রা জানান, মোস্তফা কামাল নিজ ব্যবসাপ্রতিষ্ঠান থেকে রাতে বাড়ি ফেরার পথে পৌরসভা কার্যালয়ের সামনে পৌঁছলে ৫-৬ জন সশস্ত্র দুর্বৃত্ত তাকে কুপিয়ে আহত করে। এ সময় তাদের কাছে থাকা দোকানের নগদ ৫ লাখ ৪৮ হাজার টাকা নিয়ে পালিয়ে যায়। এ সময় বাধা দিলে তার ছেলে কাজলকে (৩৫) কুপিয়ে আহত করে পালিয়ে যায় দুর্বৃত্তরা। রায়পুর থানার ওসি মোহাম্মদ লোকমান হোসেন জানান, এ ঘটনায় অভিযোগ পেয়েছি । এ ব্যাপারে প্রয়োজনীয় ব্যবস্থা নেয়া হচ্ছে।
processing...
রায়পুরে বাবা-ছেলেকে কুপিয়ে ৫ লাখ টাকা ছিনতাই
রায়পুরে রামপাল
রায়পুরে রাঙ্গুনিয়া
রায়পুরে রাউজান
রায়পুরে রামু
রায়পুরে রামগড়
রায়পুরে রায়পুর
রায়পুরে রামগঞ্জ
রায়পুরে রামগতি
রায়পুরে রাঙামাটি সদর


In [51]:
with open('E:/Dropbox/Sachetan_Crawl/201314upazilawithzila.txt', 'r', encoding = 'utf-8') as up_file:
    text = up_file.read()
with open('E:/Dropbox/Sachetan_Crawl/201314upazilawithzila_charfixed.txt', 'w', encoding = 'utf-8') as down_file:
    x = 'রায়পুর'
    print(x.replace('য়', 'য়').replace('ড়', 'ড়').replace('ৌ', 'ৌ').replace('ো', 'ো'))
    down_file.write(text.replace('য়', 'য়').replace('ড়', 'ড়').replace('ৌ', 'ৌ').replace('ো', 'ো'))

রায়পুর


In [50]:
x = 'রায়পুর'
for i in x:
    print(i)

র
া
য
়
প
ু
র
